In [3]:
# pip install statsforecast
from statsforecast import StatsForecast
from statsforecast.models import AutoARIMA
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.preprocessing import StandardScaler


In [19]:
from scipy import optimize

### Write OLSclass
class OLS:
    def __init__(self, X, y):
        self.X = X
        self.y = y
        self.beta = None

    def _loss(self, beta):
        beta = beta.reshape(-1, 1)
        self.predictions = self.X @ beta[:-1] + beta[-1] 
        residuals = self.y - self.predictions
        loss = np.sum(residuals ** 2) / (2 * len(self.y))
        return loss

    
    def fit(self):
        # Initialize coeffs to zeros
        self.beta = np.zeros((self.X.shape[1] + 1, 1))

        optimizer = optimize.minimize(fun = self._loss, x0 = self.beta.flatten(), method = "BFGS")

        print(f"Optimized Coeffs: {optimizer.x}, optimized mse: {optimizer.fun}")

# Generate artificial data
np.random.seed(42)
n = 1000
X = np.random.rand(n, 3)
y = 10 + X @ np.array([[1.5, -2.0, 3.0]]).T + np.random.randn(n, 1) * 0.5


# Fit the model
ols_model = OLS(X, y)
ols_model.fit()

Optimized Coeffs: [ 1.53188371 -1.99605682  2.96189951 10.00167429], optimized mse: 0.12590284895134896


In [21]:
df

,Total,RegionA,RegionB,StoreA1,StoreA2,StoreB1
2020-01-05,9.480604,4.522354,4.468545,2.408053,1.414072,1.475338
2020-01-12,19.139696,8.267219,7.900686,4.958850,3.743179,3.968992
2020-01-19,29.568012,13.139564,13.247396,7.403624,6.996022,7.265056
2020-01-26,39.645169,17.578957,20.758953,8.577971,8.547797,8.157899
2020-02-02,49.051358,19.649508,23.918875,10.727334,9.855547,9.792393
...,...,...,...,...,...,...
2021-10-31,964.330417,477.513880,479.851082,201.886405,170.466394,187.492354
2021-11-07,974.218909,483.200946,485.109743,201.709339,173.557255,190.029984
2021-11-14,983.413942,489.667487,490.322483,203.887816,175.094316,192.458170
2021-11-21,993.026242,493.555907,496.105357,207.316674,176.944375,194.178177


In [33]:
import pandas as pd
import numpy as np

from statsforecast.core import StatsForecast
from statsforecast.models import AutoARIMA

from hierarchicalforecast.core import HierarchicalReconciliation
from hierarchicalforecast.methods import MinTrace

# 1) Create synthetic hierarchical time series
# Hierarchy: Total -> [A, B] -> [A1, A2, B1, B2]
np.random.seed(42)
n_periods = 36

dates = pd.date_range(start='2018-01-01', periods=n_periods, freq='M')

# Bottom-level series
A1 = np.random.poisson(20, n_periods)
A2 = np.random.poisson(35, n_periods)
B1 = np.random.poisson(50, n_periods)
B2 = np.random.poisson(10, n_periods)

# Aggregate to parents
A = A1 + A2
B = B1 + B2
Total = A + B

# Combine into a DataFrame in long format
data = []
for name, series in zip(
    ['Total', 'A', 'B', 'A1', 'A2', 'B1', 'B2'],
    [Total, A, B, A1, A2, B1, B2]
):
    data.extend(zip([name]*n_periods, dates, series))

Y_df = pd.DataFrame(data, columns=['unique_id', 'ds', 'y'])

# 2) Build summing matrix S and tags manually
# S rows: all series, columns: bottom-level series
bottom_ids = ['A1', 'A2', 'B1', 'B2']
all_ids = ['Total', 'A', 'B'] + bottom_ids
S_df = pd.DataFrame(0, index=all_ids, columns=bottom_ids)
# Fill in 1's
S_df.loc['Total', :] = 1
S_df.loc['A', ['A1', 'A2']] = 1
S_df.loc['B', ['B1', 'B2']] = 1
for b in bottom_ids:
    S_df.loc[b, b] = 1

# Convert index to column 'unique_id' (required by hierarchicalforecast)
S_df = S_df.reset_index().rename(columns={'index': 'unique_id'}).astype({'A1': int, 'A2': int, 'B1': int, 'B2': int})

# Tags: dictionary with groupings
tags = {
    'levels': {
        'Total': ['Total'],
        'Level1': ['A', 'B'],
        'Bottom': bottom_ids
    }
}

# 3) Fit base forecasts with StatsForecast
sf = StatsForecast(models=[AutoARIMA(season_length=12)], freq='M', n_jobs=-1)
sf = sf.fit(Y_df)
Y_hat_df = sf.predict(h=6)  # Forecast 6 months ahead

# 4) Reconcile forecasts using MinTrace (MinT)
reconcilers = [MinTrace('ols')]
hrec = HierarchicalReconciliation(reconcilers=reconcilers)
Y_rec_df = hrec.reconcile(Y_hat_df=Y_hat_df, Y_df=Y_df, S=S_df, tags=tags)

print("Base forecasts:")
print(Y_hat_df.head())
print("\nReconciled forecasts (MinTrace):")
print(Y_rec_df.head())
print(S_df.head())
print(Y_hat_df.head())
print(tags)


/tmp/ipykernel_119442/2754002355.py:15: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(start='2018-01-01', periods=n_periods, freq='M')


Base forecasts:
  unique_id         ds  AutoARIMA
0         A 2021-01-31  51.463863
1         A 2021-02-28  55.821037
2         A 2021-03-31  55.821037
3         A 2021-04-30  55.821037
4         A 2021-05-31  55.821037

Reconciled forecasts (MinTrace):
  unique_id         ds   AutoARIMA  AutoARIMA/MinTrace_method-ols
0     Total 2021-01-31  109.573891                     108.051238
1     Total 2021-02-28  116.768364                     116.013434
2     Total 2021-03-31  116.768364                     116.392743
3     Total 2021-04-30  116.768364                     116.602048
4     Total 2021-05-31  116.768364                     116.652689
  unique_id  A1  A2  B1  B2
0     Total   1   1   1   1
1         A   1   1   0   0
2         B   0   0   1   1
3        A1   1   0   0   0
4        A2   0   1   0   0
  unique_id         ds  AutoARIMA
0         A 2021-01-31  51.463863
1         A 2021-02-28  55.821037
2         A 2021-03-31  55.821037
3         A 2021-04-30  55.821037
4         A 

/home/nitish/Documents/github/M5/.venv/lib/python3.12/site-packages/utilsforecast/processing.py:384: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  freq = pd.tseries.frequencies.to_offset(freq)
/home/nitish/Documents/github/M5/.venv/lib/python3.12/site-packages/utilsforecast/processing.py:440: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  freq = pd.tseries.frequencies.to_offset(freq)


In [36]:
Y_hat_df.rename({"AutoARIMA": "separateForecast"}, axis = 1, inplace = True)
Y_hat_df['ReconciledForecast'] = Y_rec_df['MinTrace']
Y_hat_df

KeyError: 'MinTrace'

In [40]:
Y_rec_df.loc[Y_rec_df.unique_id.isin(['Total', 'A', 'B', 'A1', 'A2', 'B1', 'B2'])&(Y_rec_df.ds == "2021-01-31"), :]

,unique_id,ds,AutoARIMA,AutoARIMA/MinTrace_method-ols
0,Total,2021-01-31,109.573891,108.051238
6,A,2021-01-31,51.463863,53.132099
12,B,2021-01-31,52.821770,54.919139
18,A1,2021-01-31,19.805555,19.659972
24,A2,2021-01-31,33.617710,33.472127
30,B1,2021-01-31,46.124126,45.549410
36,B2,2021-01-31,9.944445,9.369728


In [42]:
53.132099	+ 54.919139

108.051238